<a href="https://colab.research.google.com/github/mgonzal1/nma_wombat_connectivy_project/blob/Eve/connectivity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#@title import matplotlib and set defaults
from matplotlib import rcParams 
from matplotlib import pyplot as plt
import numpy as np
rcParams['figure.figsize'] = [20, 4]
rcParams['font.size'] =15
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False
rcParams['figure.autolayout'] = True

In [7]:
#@title Data retrieval
import os, requests

fname = []
for j in range(3):
  fname.append('steinmetz_part%d.npz'%j)
url = ["https://osf.io/agvxh/download"]
url.append("https://osf.io/uv3mw/download")
url.append("https://osf.io/ehmw2/download")

for j in range(len(url)):
  if not os.path.isfile(fname[j]):
    try:
      r = requests.get(url[j])
    except requests.ConnectionError:
      print("!!! Failed to download data !!!")
    else:
      if r.status_code != requests.codes.ok:
        print("!!! Failed to download data !!!")
      else:
        with open(fname[j], "wb") as fid:
          fid.write(r.content)



alldat = np.array([])
for j in range(len(fname)):
  alldat = np.hstack((alldat, np.load('steinmetz_part%d.npz'%j, allow_pickle=True)['dat']))

In [9]:
n_sessions = 39
trial_time = 2.5
all_dat = np.zeros(None,None)
n_trials_session = []
n_neurons_session = []
fr = []
for session in range(n_sessions):
  temp = alldat[session]['spks'][:,:,:].sum(axis=2)/trial_time
  n_neurons, n_trials = temp.shape
  fr.append(temp)
  n_trials_session.append(n_trials)
  n_neurons_session.append(n_neurons) 
n_trials_session = np.array(n_trials_session)
n_neurons_session = np.array(n_neurons_session)

In [8]:
def get_trainning_set():
  """
  Get a subset of alldat for trainning purposes.

  Returns:
    map: dat['spks']: neurons by trials by time bins. Time bin = 10ms.
         dat['brain_area']: brain area for each neuron recorded.
         dat['contrast_right']: contrast level for the right stimulus, which is always contralateral to the recorded brain areas.
         dat['contrast_left']: contrast level for left stimulus.
         dat['response']: which side the response was (-1, 0, 1). Choices for the right stimulus are -1.
         dat['response_times']: when the response was registered, which has to be after the go cue. The mouse can turn the wheel before the go cue (and always does!).
         dat['wheel']: exact position of the wheel that the mice uses to make a response, binned at 10ms.
         dat['pupil']: pupil area (noisy, because pupil is very small).
         dat['lfp']: recording of the local field potential in each brain area from this experiment, binned at 10ms.
         dat['brain_area_lfp']: brain area names for the LFP channels.
  """
  # Functions to split data and get a subset. If condition of areas  
  tranning_set = alldat
  return trainning_set

def get_validation_set():
  """
  Get a subset of alldat for validation purposes. This should be ~5%-10% of all the data.

  Returns:
    map: dat['spks']: neurons by trials by time bins. Time bin = 10ms.
         dat['brain_area']: brain area for each neuron recorded.
         dat['contrast_right']: contrast level for the right stimulus, which is always contralateral to the recorded brain areas.
         dat['contrast_left']: contrast level for left stimulus.
         dat['response']: which side the response was (-1, 0, 1). Choices for the right stimulus are -1.
         dat['response_times']: when the response was registered, which has to be after the go cue. The mouse can turn the wheel before the go cue (and always does!).
         dat['wheel']: exact position of the wheel that the mice uses to make a response, binned at 10ms.
         dat['pupil']: pupil area (noisy, because pupil is very small).
         dat['lfp']: recording of the local field potential in each brain area from this experiment, binned at 10ms.
         dat['brain_area_lfp']: brain area names for the LFP channels.
  """
  # Functions to split data and get a subset 
  validation_set = alldat
  return validation_set

def get_visual_spikes(data_set):
  return

def get_motor_spikes(data_set):
  return

def get_stimulus(data_set):
  return

def get_decision(data_set):
  return
